In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from trace_process import *

In [2]:
MTU = 1514
MB = 1024 * 1024
KB = 1024
GB = 1024 * 1024 * 1024
SECONDS = 1e9
MILLI_SECONDS = 1e6
MICRO_SECONDS = 1e3
NANO_SECONDS = 1
IPS = [212, 237]
MASTER = 212
TARGET_IP = 237

PACKETS = "../data/40/node-3/train/packets"
ALLOCS = "../data/40/node-3/train/allocations"

In [3]:
RES = 200
TIMERES = 'Milliseconds'
TIMERESL = 'm'

In [4]:
def get_packets(file_path, timefactor=SECONDS, volumefactor=KB):
    df = pd.read_csv(
        file_path, 
        header=None,
        index_col=False,
        names=['timestamp', 'size', 'src_ip', 'dest_ip', 'src_port', 'dest_port', 'dir'], 
        dtype={"dir": "int8"},
    )
    # df = df[df['dir'] == 1]
    #df['timestamp'] = (df['timestamp'] - df['timestamp'].min()) / timefactor
    df['size'] = df['size'] / volumefactor
    df = df.sort_values(by='timestamp')
    return df

In [5]:
def get_trace(file_path, timefactor=SECONDS, volumefactor=KB):
    df = pd.read_csv(
        file_path, 
        header=None,
        index_col=False,
        names=['timestamp', 'size'],
        sep='\t'
    )
    #df['timestamp'] = (df['timestamp'] - df['timestamp'].min()) / timefactor
    df['size'] = df['size'] / volumefactor
    df = df.sort_values(by='timestamp')
    return df

In [6]:
df = get_packets(PACKETS, volumefactor=KB, timefactor=MILLI_SECONDS)
df = df[['timestamp', 'size', 'src_ip', 'dest_ip']]
df = df[(df['src_ip'] == MASTER) & (df['dest_ip'] == TARGET_IP)]

In [7]:
df_alloc = get_trace(ALLOCS, volumefactor=KB, timefactor=MILLI_SECONDS)

In [9]:
ts = df['timestamp'].values
vals = df['size'].values
ts_alloc = df_alloc['timestamp'].values
vals_alloc = df_alloc['size'].values
y = []
ya = []
x = []
start, end = ts[0], ts[0] + RES
while end < ts[-1]:
    in_ts = np.argwhere((start <= ts) & (ts <= end))
    in_ts_alloc = np.argwhere((start <= ts_alloc) & (ts_alloc <= end))
    x.append(start)
    if len(in_ts) > 1:
        new_s, new_end = in_ts[0][0], in_ts[-1][0]
        y.append(np.sum(vals[new_s:new_end]))
        x.append(start)
        ts = ts[in_ts[-1][0]:]
        vals = vals[in_ts[-1][0]:]
    else:
        y.append(0)
    if len(in_ts_alloc) > 1:
        new_s_alloc, new_end_alloc = in_ts_alloc[0][0], in_ts_alloc[-1][0]
        ya.append(np.sum(vals_alloc[new_s:new_end]))
        ts_alloc = ts_alloc[in_ts[-1][0]:]
        vals_alloc = vals_alloc[in_ts[-1][0]:]
    else:
        ya.append(0)
        
    start, end = start + RES, end + RES

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(50, 10))
plt.title(f"Res: {RES} {TIMERES} {MASTER} -> {TARGET_IP}")
plt.xlabel(f"Time ({TIMERESL})")
plt.ylabel("Data, KB")
plt.plot(x, y)

In [ ]:
plt.figure(figsize=(50, 10))
plt.title(f"Res: {RES} {TIMERES} {MASTER} -> {TARGET_IP}")
plt.xlabel(f"Time ({TIMERESL})")
plt.ylabel("Data, KB")
plt.plot(x, ya)

In [8]:
df = pd.DataFrame({"timestamp": x, "size": y})
df.to_csv('alexnet-200-tf.csv', index=False)
# ts = pd.read_csv('gpt-200-pytorch.csv')


In [9]:
for c in df.columns:
    mean = df[c].mean()
    stdev = df[c].std()

    df[c] = (df[c] - mean) / stdev
    df[c] = (df[c] - mean) / stdev

In [10]:
"""
ts = df['timestamp'].values
vals = df['size'].values
y = np.empty(0)
x = np.empty(0)
times = np.arange(ts[0], ts[-1], RES)
times = np.column_stack((times[:-1], times[1:]))
fw = 100
while True:
    tmp_times = times[:fw]
    times = times[fw:]
    a = np.logical_and(tmp_times[:, 0][:, None] < ts, ts < tmp_times[:, 1][:, None])
    x = np.append(x, (a * vals).sum(axis=1))
    y = np.append(y, tmp_times[:, 0])
    if len(times) < fw:
        break
plt.figure(figsize=(50, 10))
plt.title(f"Res: {RES} {TIMERES} {MASTER} -> {TARGET_IP}")
plt.xlabel(f"Time ({TIMERESL})")
plt.ylabel("Data, KB")
plt.plot(x, y)
"""

'\nts = df[\'timestamp\'].values\nvals = df[\'size\'].values\ny = np.empty(0)\nx = np.empty(0)\ntimes = np.arange(ts[0], ts[-1], RES)\ntimes = np.column_stack((times[:-1], times[1:]))\nfw = 100\nwhile True:\n    tmp_times = times[:fw]\n    times = times[fw:]\n    a = np.logical_and(tmp_times[:, 0][:, None] < ts, ts < tmp_times[:, 1][:, None])\n    x = np.append(x, (a * vals).sum(axis=1))\n    y = np.append(y, tmp_times[:, 0])\n    if len(times) < fw:\n        break\nplt.figure(figsize=(50, 10))\nplt.title(f"Res: {RES} {TIMERES} {MASTER} -> {TARGET_IP}")\nplt.xlabel(f"Time ({TIMERESL})")\nplt.ylabel("Data, KB")\nplt.plot(x, y)\n'

In [11]:
from xgboost import XGBRegressor, DMatrix, train
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error, r2_score,
    mean_absolute_error, accuracy_score,
    f1_score, roc_auc_score,
    log_loss
)

In [12]:
def prepare_files(df, window_size, target_column='size'):
    target = df[target_column]
    final_df = df.copy()
    for sample_num in range(1, window_size):
        shifted = df.shift(sample_num)
        shifted.columns = map(lambda x: x+str(sample_num), shifted.columns)
        final_df = pd.concat([shifted, final_df], axis=1)
    final_df = final_df.fillna(0)
    final_df = final_df.drop(target_column, axis=1)
    return final_df, target

In [13]:
def eval_metrics(real, prediction):
    mse = mean_squared_error(real, prediction)
    mae = mean_absolute_error(real, prediction)
    r2 = r2_score(real, prediction)

    return {'mse': mse, 'mae': mae, 'r2': r2}

In [14]:
def eval_model(model, xtest, ytest):
    y_pred = model.predict(xtest)
    return eval_metrics(ytest, y_pred)

In [15]:
def xgboost_learn(xtrain, ytrain, xtest, ytest):
    model = XGBRegressor(objective='reg:squarederror', 
                         n_estimators=1000, 
                         max_depth=10, 
                         eval_metric=r2_score, 
                         booster='gbtree',
                         predictor="gpu_predictor",
                         tree_method='gpu_hist',
                        )
    model.fit(xtrain, ytrain)
    result = {}
    result['train'] = eval_model(model, xtrain, ytrain)
    result['test'] = eval_model(model, xtest, ytest)

    return result, model

In [16]:
inputs, outputs = prepare_files(df, 50)
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.2, shuffle=False)
xgboost_learn(x_train, y_train, x_test, y_test)

({'train': {'mse': 60.79750838143415,
   'mae': 1.4129572290306474,
   'r2': 0.31010228808286733},
  'test': {'mse': 72.8722138408932,
   'mae': 1.7878457822723037,
   'r2': 0.013655265500170821}},
 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=<function r2_score at 0x7fb4d9af0670>, gamma=0,
              gpu_id=0, grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=10, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              predictor='gpu_predictor', random_state=0, reg_alpha=0,
              reg_lambda=1, ...))

RES = 2000
({'train': {'mse': 2213857.391212464,
   'mae': 730.6404319857144,
   'r2': 0.7279064089400382},
  'test': {'mse': 2693812.8702931516,
   'mae': 815.724213257461,
   'r2': 0.6574755396638999}},

RES = 200 ResNet NCCL
'train': {'mse': 9.101269742950386e-07,
  
({'train': {'mse': 2.9009046081819423e-07,
   'mae': 0.00011134981196798435,
   'r2': 0.7098828104436443},
  'test': {'mse': 3.43119514130418e-07,
   'mae': 0.00012053085141676116,
   'r2': 0.6566614615364892}}

RES = 200 GPT C20D
({'train': {'mse': 3.222166312277829e-07,
   'mae': 0.00012348057231785167,
   'r2': 0.6816371602112872},
  'test': {'mse': 3.7919567986376153e-07,
   'mae': 0.0001340097186057974,
   'r2': 0.6124819743566556}}

In [28]:
split = 0.2
target = 'target'
features = ['timestamp', 'size']
ts = pd.read_csv('alexnet-200-tf.csv')
ts = ts.apply(pd.to_numeric)
ts[target] = ts['size'].shift(-1)
ts = ts[:-1]
split_index = int(ts.shape[0] * split)
df_train = ts[:-split_index]
df_test = ts[-split_index:]

In [29]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

for c in df_train.columns:
    mean = df_train[c].mean()
    stdev = df_train[c].std()

    df_train[c] = (df_train[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev

/tmp/ipykernel_100603/362411511.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[c] = (df_train[c] - mean) / stdev
/tmp/ipykernel_100603/362411511.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[c] = (df_test[c] - mean) / stdev


In [30]:
import torch
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[target].values).float()
        self.X = torch.tensor(dataframe[features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

In [21]:
from torch.utils.data import DataLoader
torch.manual_seed(99)

train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)

X, y = next(iter(train_loader))
print(X.shape)
print(X)

torch.Size([3, 50, 2])
tensor([[[ 0.5489, -0.1164],
         [ 0.5489, -0.1164],
         [ 0.5489, -0.1164],
         [ 0.5489, -0.1164],
         [ 0.5489, -0.1164],
         [ 0.5490, -0.1164],
         [ 0.5490, -0.1164],
         [ 0.5490, -0.1164],
         [ 0.5490, -0.1164],
         [ 0.5491, -0.1164],
         [ 0.5491, -0.1164],
         [ 0.5491, -0.1164],
         [ 0.5491, -0.1164],
         [ 0.5491, -0.1164],
         [ 0.5492, -0.1164],
         [ 0.5492, -0.1164],
         [ 0.5492, -0.1164],
         [ 0.5492, -0.1164],
         [ 0.5492, -0.1164],
         [ 0.5493, -0.1164],
         [ 0.5493, -0.1164],
         [ 0.5493, -0.1164],
         [ 0.5493,  2.2523],
         [ 0.5494, -0.1164],
         [ 0.5494, -0.1164],
         [ 0.5494, -0.1164],
         [ 0.5494, -0.1164],
         [ 0.5494, -0.1164],
         [ 0.5495, -0.1164],
         [ 0.5495, -0.1164],
         [ 0.5495, -0.1164],
         [ 0.5495, -0.1164],
         [ 0.5496, -0.1164],
         [ 0.5496, -

In [31]:
torch.manual_seed(101)

batch_size = 128
sequence_length = 128

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([128, 128, 2])
Target shape: torch.Size([128])


In [32]:
from torch import nn

class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_features, hidden_units):
        super().__init__()
        self.num_features = num_features  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

In [33]:
learning_rate = 5e-5
num_hidden_units = 64

model = ShallowRegressionLSTM(num_features=len(features), hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")

def test_model(data_loader, model, loss_function):
    
    num_batches = len(data_loader)
    total_loss = 0
    y_pred = []
    y_real = []

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()
            y_pred.extend(output.numpy())
            y_real.extend(y.numpy())

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")
    print(f"R2 score: {r2_score(y_real, y_pred)}")

In [35]:
print("Untrained test\n--------")
test_model(test_loader, model, loss_function)
print()

for ix_epoch in range(50):
    print(f"Epoch {ix_epoch}\n---------")
    train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_model(test_loader, model, loss_function)
    print()

Untrained test
--------
Test loss: 0.8405627168801565
R2 score: -0.0025698171718053775

Epoch 0
---------
Train loss: 0.9980794819581579
Test loss: 0.8350524965536374
R2 score: 0.004003376221292654

Epoch 1
---------
Train loss: 0.994266715877576
Test loss: 0.8330200204912287
R2 score: 0.006428917011861546

Epoch 2
---------
Train loss: 0.9912460904832607
Test loss: 0.830978667385912
R2 score: 0.008864562717770919

Epoch 3
---------
Train loss: 0.9880021697698305
Test loss: 0.8294705074229756
R2 score: 0.01066398184083761

Epoch 4
---------
Train loss: 0.9856897299155557
Test loss: 0.8278548217718847
R2 score: 0.012591146990837188

Epoch 5
---------
Train loss: 0.9830168916460258
Test loss: 0.8268331653581789
R2 score: 0.013809372292562516

Epoch 6
---------
Train loss: 0.9820899416891058
Test loss: 0.8262300617121255
R2 score: 0.014529081930570897

Epoch 7
---------
Train loss: 0.9805930501882895
Test loss: 0.8259279870469936
R2 score: 0.01488935750589182

Epoch 8
---------
Train loss


KeyboardInterrupt



In [ ]:
test_model(test_loader, model, loss_function)

RES = 200 ResNet NCCL
Test loss: 0.6859737798425152
R2 score: 0.3139429240444592

RES = 200 GPT C20D
Test loss: 0.3235129721796335
R2 score: 0.6663728481527116

In [36]:
((df['size'] == 0) * 1).sum() / len(df)

0.9124033429664923